# Thresholds: Average number of alerts (monthly)

# Alerts overview

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import glob

import warnings
warnings.filterwarnings("ignore")

In [2]:
#load output files

def determine_number_of_models(folder_to_search):
    folders = [f for f in glob.glob(folder_to_search + "**/", recursive = True)]
    model_location = []
    model_names = []
    for folderscounter in folders:
        model_location = [f for f in glob.glob(folderscounter + "/*tst_pred*", recursive = True)]
    if len(model_location) > 0:
        for cnt in model_location:
            split_str = cnt.split('\\')
            model_names.append(split_str[-1][:-4])
    
    return len(model_location), model_location, model_names


In [3]:
def load_csv(model):
    dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    dfoutput = pd.read_csv(model, parse_dates = ['timestamp'], date_parser = dateparse)
    dfoutput.set_index(['timestamp'], inplace = True)
    return dfoutput


In [4]:
def create_model_ids(models):
    mid = []
    for i in range(0, no_of_models):
        m = 'M' + str(i+1)
        mid.append(m)
    return mid

In [5]:
def create_new_columns(df, model_name):
    df.prediction = df.prediction.replace(1, 0)
    df.prediction = df.prediction.replace(-1, 1)

    df['Anomalies'] = 0
    df.loc[(df['system_status'] == 'online') & (df['prediction'] == 1), 'Anomalies'] = 1

    df['Alerts'] = 0
    df.loc[(df['system_status'] == 'online') & (df['alert'] == 1) & (df['prediction'] == 1), 'Alerts'] = 1
    
    return df

def average_alerts(model_name):
    avg_alerts = []
    for cnt in model_location:
        df = load_csv(cnt)
        sampdf = create_new_columns(df, model_name)
        sampdf = sampdf[['Anomalies','Alerts']].resample('M').sum()
        xx = round(np.mean(sampdf['Alerts']), 2)
        avg_alerts.append(xx)
    return avg_alerts


In [6]:
def results(model_name, avg_alerts):
    mid = create_model_ids(no_of_models)
    output_df = pd.DataFrame()
    for i in range(0, no_of_models):
        output_df.loc[i, 'ModelIds'] = mid[i]
        output_df.loc[i, 'Models'] = model_names[i]
        output_df.loc[i, 'Average Alerts'] = aa[i]
        if aa[i] <= 15:
            output_df.loc[i, 'Below Threshold'] = 'Y'
        else:
            output_df.loc[i, 'Below Threshold'] = 'N'
            
    return output_df


# GT5

In [7]:
folder_to_search = r'C:/Users/Ranja.Sarkar/PAMre/KPIs/BSP_GT5/QualitativeKPI'
no_of_models, model_location, model_names = determine_number_of_models(folder_to_search)


In [8]:
k = 0
for cnt in model_location:
    print('processing', model_names[k])
    aa = average_alerts(model_names[k])
    analysis_result_df = results(model_names[k], aa)
    k = k+1

processing Brunei_Champion_GT5_tst_pred_w1d_m4hr_svm_nu0.01_krnrbf_gam0.01_01Aug2018_01Jul2019
processing Brunei_Champion_GT5_tst_pred_w1d_m4hr_svm_nu0.01_krnrbf_gam0.03_01Aug2018_01Jul2019
processing Brunei_Champion_GT5_tst_pred_w1d_m4hr_svm_nu0.02_krnrbf_gam0.01_01Aug2018_01Jul2019
processing Brunei_Champion_GT5_tst_pred_w1d_m4hr_svm_nu0.02_krnrbf_gam0.03_01Aug2018_01Jul2019


In [10]:
#analysis_result_df.to_csv('avg_monthly_alerts_GT5.csv')

# LPC4

In [7]:
folder_to_search = r'C:/Users/Ranja.Sarkar/PAMre/KPIs/BSP_GT5/QualitativeKPI/LPC4'
no_of_models, model_location, model_names = determine_number_of_models(folder_to_search)

In [8]:
k = 0
for cnt in model_location:
    print('processing', model_names[k])
    aa = average_alerts(model_names[k])
    analysis_result_df = results(model_names[k], aa)
    k = k+1

processing NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.001_krnrbf_gam0.001_01May2019_20Mar2020
processing NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.001_krnrbf_gam0.05_01May2019_20Mar2020
processing NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.01_krnrbf_gam0.001_01May2019_20Mar2020
processing NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.05_krnrbf_gam0.001_01May2019_20Mar2020
processing NL_ONEGAS_LPC4_tst_pred_w1d_m1hr_svm_nu0.05_krnrbf_gam0.05_01May2019_20Mar2020
processing NL_ONEGAS_LPC4_tst_pred_w1d_m4hr_svm_nu0.01_krnrbf_gam0.001_01May2019_20Mar2020


In [ ]:
#analysis_result_df.to_csv('avg_monthly_alerts_LPC4.csv')